In [1]:
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as la
## Construct LSTM using Tensorflow + Keras
# Import Libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import SimpleRNN
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras import optimizers


######################################################################
######################################################################
######################################################################
## CSP File from: https://github.com/spolsley/common-spatial-patterns

# CSP takes any number of arguments, but each argument must be a collection of trials associated with a task
# That is, for N tasks, N arrays are passed to CSP each with dimensionality (# of trials of task N) x (feature vector)
# Trials may be of any dimension, provided that each trial for each task has the same dimensionality,
# otherwise there can be no spatial filtering since the trials cannot be compared
def CSP(*tasks):
	if len(tasks) < 2:
		print("Must have at least 2 tasks for filtering.")
		return (None,) * len(tasks)
	else:
		filters = ()
		# CSP algorithm
		# For each task x, find the mean variances Rx and not_Rx, which will be used to compute spatial filter SFx
		iterator = range(0,len(tasks))
		for x in iterator:
			# Find Rx
			Rx = covarianceMatrix(tasks[x][0])
			for t in range(1,len(tasks[x])):
				Rx += covarianceMatrix(tasks[x][t])
			Rx = Rx / len(tasks[x])

			# Find not_Rx
			count = 0
			not_Rx = Rx * 0
			for not_x in [element for element in iterator if element != x]:
				for t in range(0,len(tasks[not_x])):
					not_Rx += covarianceMatrix(tasks[not_x][t])
					count += 1
			not_Rx = not_Rx / count

			# Find the spatial filter SFx
			SFx = spatialFilter(Rx,not_Rx)
			filters += (SFx,)

			# Special case: only two tasks, no need to compute any more mean variances
			if len(tasks) == 2:
				filters += (spatialFilter(not_Rx,Rx),)
				break
		return filters

# covarianceMatrix takes a matrix A and returns the covariance matrix, scaled by the variance
def covarianceMatrix(A):
	Ca = np.dot(A,np.transpose(A))/np.trace(np.dot(A,np.transpose(A)))
	return Ca

# spatialFilter returns the spatial filter SFa for mean covariance matrices Ra and Rb
def spatialFilter(Ra,Rb):
	R = Ra + Rb
	E,U = la.eig(R)

	# CSP requires the eigenvalues E and eigenvector U be sorted in descending order
	ord = np.argsort(E)
	ord = ord[::-1] # argsort gives ascending order, flip to get descending
	E = E[ord]
	U = U[:,ord]

	# Find the whitening transformation matrix
	P = np.dot(np.sqrt(la.inv(np.diag(E))),np.transpose(U))

	# The mean covariance matrices may now be transformed
	Sa = np.dot(P,np.dot(Ra,np.transpose(P)))
	Sb = np.dot(P,np.dot(Rb,np.transpose(P)))

	# Find and sort the generalized eigenvalues and eigenvector
	E1,U1 = la.eig(Sa,Sb)
	ord1 = np.argsort(E1)
	ord1 = ord1[::-1]
	E1 = E1[ord1]
	U1 = U1[:,ord1]

	# The projection matrix (the spatial filter) may now be obtained
	SFa = np.dot(np.transpose(U1),P)
	return SFa.astype(np.float32)
######################################################################
def GetMinSteps(indeces, data):
	minVal = 9999;
	for index in indeces:
		length = data[index].shape[1];
		if(length < minVal):
			minVal = length;
	return minVal
######################################################################
def GetData(indeces, dataIn, truncateValue):
	dataOut = []
	truncate = True;
	if truncateValue == 0:
		truncate = False;
	for idx in indeces:
		currentData = dataIn[idx]
		if truncate:
			dataOut.append(currentData[:,0:truncateValue])
		else:
			dataOut.append(currentData)
	return np.asarray(dataOut)
######################################################################
######################################################################
######################################################################
enableDropout = True;
dropoutPercentage = 0.3;

fileNames = ['CLASubjectA1601083StLRHand.mat',\
             'CLASubjectB1510193StLRHand.mat',\
             'CLASubjectB1510203StLRHand.mat',\
             'CLASubjectB1512153StLRHand.mat',\
             'CLASubjectC1511263StLRHand.mat',\
             'CLASubjectC1512163StLRHand.mat',\
             'CLASubjectC1512233StLRHand.mat',\
             'CLASubjectD1511253StLRHand.mat',\
             'CLASubjectE1512253StLRHand.mat',\
             'CLASubjectE1601193StLRHand.mat',\
             'CLASubjectE1601223StLRHand.mat',\
             'CLASubjectF1509163StLRHand.mat',\
             'CLASubjectF1509173StLRHand.mat',\
             'CLASubjectF1509283StLRHand.mat'];

numDatasets = len(fileNames);

testAccuracyNoCSP = np.zeros((1,numDatasets))
trainAccuracyNoCSP = np.zeros((1,numDatasets))
testAccuracyWithCSP = np.zeros((1,numDatasets))
trainAccuracyWithCSP = np.zeros((1,numDatasets))

for dataset in range(0, numDatasets):

	fileName = fileNames[dataset];

	file = sio.loadmat('../../../matDown/CLA_Data/{}'.format(fileName)) #replace with .mat file name
	header=file['__header__']
	version=file['__version__']
	glob=file['__globals__']
	#ans=file['ans']


	#x=file['x']
	o=file['o'][0][0]
	data=o['data']
	data = np.transpose(data)
	data = data[0:21,:];
	print(data)
	nS=o['nS'][0][0]
	#values of structure seem to be 2D numpy arrays, if originally a scalar in Matlab.
	#use [0][0] to get scalar.
	#print("Number of samples: {numSamples}".format(numSamples=nS))
	test=o['id'][0] #id value became a 1D array of size 1 for some reason. use [0] to get value
	#print("Dataset ID: {id}".format(id=test))
	chnames=o['chnames'][:,0] #[:,0] converts from 2D array back to 1D array
	#print("Channel names: {channelNames}".format(channelNames=chnames))
	markers = o['marker']
	## The markers are all still individual arrays of size 1x1, so we convert them to an array with single values
	markersArray = []
	for marker in markers:
		markersArray.append(marker[0])
	markersArray = np.asarray(markersArray)

	#################################
	#################################
	#5F interaction paradigm
	#1-thumb MI, 2-index finger MI, 3-middle finger MI, 4-ring finger MI, 5-pinkie finger MI

	#all paradigms
	#99-initial relaxation period
	#91-inter-session rest break period
	#92-experiment end
	#################################

	## Find the starting indeces where the marker changes
	changeIdxs = np.where(np.transpose(markersArray)[:-1] != np.transpose(markersArray)[1:])[0]
	#print("Number of index changes: {idxChanges}".format(idxChanges=changeIdxs.shape[0]))
	## Split the data so that it has its matching marker
	dataSplit = np.array_split(data, changeIdxs[:-1], axis=1)
	splitCount = 0
	for splitData in dataSplit:
		splitCount += 1
	#print("Number of arrays in data split: {num}".format(num=splitCount))
	## Retrieve the marker values for each of the change indeces (changeIdxs)
	markerTargets = markersArray[changeIdxs];
	#print("Number of marker targets: {numTargets}".format(numTargets=markerTargets.shape[0]))

	## To Apply CSP, we first only get the indeces for MI tasks 1 and 2 (left and right hand, respectively.)
	lhIdx = np.where(markerTargets == 1)[0]
	rhIdx = np.where(markerTargets == 2)[0]
	#llIdx = np.where(markerTargets == 4)[0]
	#tIdx = np.where(markerTargets == 5)[0]
	#rlIdx = np.where(markerTargets == 6)[0]
	
	
	
	
	
	#tCount = tIdx.shape
	#print("Thumb Marker Count: {}\tSize of First: ({},{})".format(tCount, dataSplit[tIdx[0]].shape[0],dataSplit[tIdx[0]].shape[1]))
	#print("Thumb Marker Count: {}\tSize of Second: ({},{})".format(tCount, dataSplit[tIdx[1]].shape[0],dataSplit[tIdx[1]].shape[1]))
	#print("Thumb Marker Count: {}\tSize of Third: ({},{})".format(tCount, dataSplit[tIdx[2]].shape[0],dataSplit[tIdx[2]].shape[1]))


	lhIdxMin = GetMinSteps(lhIdx, dataSplit)
	rhIdxMin = GetMinSteps(rhIdx, dataSplit)
	#llIdxMin = GetMinSteps(llIdx, dataSplit)
	#tIdxMin = GetMinSteps(tIdx, dataSplit)
	#rlIdxMin = GetMinSteps(rlIdx, dataSplit)
	#minValues = [lhIdxMin, rhIdxMin, llIdxMin, tIdxMin, rlIdxMin]
	minValues = [lhIdxMin, rhIdxMin]
	#minValues

	#Truncate the data to the min size
	minValue = np.min(minValues)
	print(minValue)

	lhData = GetData(lhIdx, dataSplit, minValue)
	rhData = GetData(rhIdx, dataSplit, minValue)
	#llData = GetData(llIdx, dataSplit, minValue)
	#tData = GetData(tIdx, dataSplit, minValue)
	#rlData = GetData(rlIdx, dataSplit, minValue)
	#print("Length of tData: {}".format(len(tData)))
	#print("Length of iData: {}".format(len(iData)))
	#print("Length of mData: {}".format(len(mData)))
	#print("Length of rData: {}".format(len(rData)))
	#print("Length of pData: {}".format(len(pData)))

	#minLen = np.min([len(lhData), len(rhData), len(llData), len(tData), len(rlData)])
	minLen = np.min([len(lhData), len(rhData)])
	##Want to make sure they are balanced, so we keep the minLen values
	lhData = lhData[0:minLen]
	rhData = rhData[0:minLen]
	#llData = llData[0:minLen]
	#tData = tData[0:minLen]
	#rlData = rlData[0:minLen]


	# In[68]:


	#Construct the target arrays and merge the data
	lhTargets = np.tile(np.array([1,0]),(minLen,1))
	rhTargets = np.tile(np.array([0,1]),(minLen,1))
	#llTargets = np.tile(np.array([0,0,1,0,0]),(minLen,1))
	#tTargets = np.tile(np.array([0,0,0,1,0]),(minLen,1))
	#rlTargets = np.tile(np.array([0,0,0,0,1]),(minLen,1))
    
	#Print the sizes of each data
	#print("lhData shape: {}".format(lhData.shape))

	#markerTargets = np.vstack((lhTargets, rhTargets, llTargets, tTargets, rlTargets))
	markerTargets = np.vstack((lhTargets, rhTargets))
	#Data = np.vstack((lhData, rhData, llData, tData, rlData))
	Data = np.vstack((lhData, rhData))
    
	#Sanity Check
	#print("Data Shape: {arg1}\tmarkerTargets Shape: {arg2}".format(arg1=Data.shape, arg2=markerTargets.shape))

	## Shuffle the data
	Data, markerTargets = shuffle(Data, markerTargets, random_state=0)

	## Split into train and test sets
	DataTrain, DataTest, markerTargetsTrain, markerTargetsTest = train_test_split(Data, markerTargets, test_size=0.3, random_state=1)
	markerTargetsTrain.shape

	## Reshape the data for time-series processing
	## Syntax np.reshape((numExamples, numTimeSteps, numInputs/numFeatures))
	DataTrainRe = DataTrain.reshape((DataTrain.shape[0], DataTrain.shape[1], DataTrain.shape[2]))
	DataTestRe = DataTest.reshape((DataTest.shape[0], DataTest.shape[1], DataTest.shape[2]))

	## Construct the model
	LSTM_EEG = Sequential()
	LSTM_EEG.add(LSTM((100),batch_input_shape=(None,DataTrainRe.shape[1], DataTrainRe.shape[2]), return_sequences=True))
	if(enableDropout):
		LSTM_EEG.add(Dropout(dropoutPercentage))
	LSTM_EEG.add(LSTM((50), return_sequences=False))
	if(enableDropout):
		LSTM_EEG.add(Dropout(dropoutPercentage))
	LSTM_EEG.add(Dense((2),activation='sigmoid'))

	LSTM_EEG.summary()
	sgd = optimizers.SGD(lr=0.05, decay=1e-6, momentum=0.9, nesterov=True)
	LSTM_EEG.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])


	# In[79]:


	history = LSTM_EEG.fit(DataTrain, markerTargetsTrain, epochs=30,verbose=2, batch_size=16)


	# In[80]:


	predictionsTest = LSTM_EEG.predict(DataTest)


	# In[81]:


	predictionsTest[predictionsTest>0.5] = 1


	# In[82]:


	predictionsTest[predictionsTest <= 0.5] = 0


	# In[83]:


	comparisonArrayTest = predictionsTest == markerTargetsTest


	# In[85]:


	correctCountTest = 0
	for boolValues in comparisonArrayTest:
		if(boolValues[0] & boolValues[1]):
			correctCountTest += 1
	falseCountTest = DataTest.shape[0] - correctCountTest

	predictionsTrain = LSTM_EEG.predict(DataTrain)
	predictionsTrain[predictionsTrain>0.5] = 1;
	predictionsTrain[predictionsTrain<=0.5] = 0;
	comparisonArrayTrain = predictionsTrain == markerTargetsTrain;

	correctCountTrain = 0
	for boolValues in comparisonArrayTrain:
		if(boolValues[0] & boolValues[1]):
			correctCountTrain += 1
	falseCountTrain = DataTrain.shape[0] - correctCountTrain


	# In[87]:
	trainAcc_noCSP = (correctCountTrain*100/DataTrain.shape[0]);
	testAcc_noCSP = (correctCountTest*100/DataTest.shape[0]);

	#Save these values into the array
	trainAccuracyNoCSP[0,dataset] = trainAcc_noCSP;
	testAccuracyNoCSP[0, dataset] = testAcc_noCSP;

	print("#################################")
	print("#################################")
	print("Training Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, trainAcc_noCSP))
	print("#################################")
	print("#################################")
	print("Testing Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, testAcc_noCSP))
	print("#################################")
	print("#################################")


	# In[88]:


	## Applying CSP to 5F data
	#filters = CSP(lhData, rhData, llData, tData, rlData)
	filters = CSP(lhData, rhData)

	# In[90]:


	filtersArray = np.asarray(filters)


	# In[91]:


	filtersArray.shape


	# In[92]:


	lhData_CSP = np.matmul(np.transpose(filtersArray[0]), lhData)
	rhData_CSP = np.matmul(np.transpose(filtersArray[1]), rhData)
	#llData_CSP = np.matmul(np.transpose(filtersArray[2]), llData)
	#tData_CSP = np.matmul(np.transpose(filtersArray[3]), tData)
	#rlData_CSP = np.matmul(np.transpose(filtersArray[4]), rlData)


	# In[93]:


	#Data_CSP = np.vstack((lhData_CSP, rhData_CSP, llData_CSP, tData_CSP, rlData_CSP))
	Data_CSP = np.vstack((lhData_CSP, rhData_CSP))


	# In[94]:


	#Construct the target arrays and merge the data
	lhTargets = np.tile(np.array([1,0]),(minLen,1))
	rhTargets = np.tile(np.array([0,1]),(minLen,1))
	#llTargets = np.tile(np.array([0,0,1,0,0]),(minLen,1))
	#tTargets = np.tile(np.array([0,0,0,1,0]),(minLen,1))
	#rlTargets = np.tile(np.array([0,0,0,0,1]),(minLen,1))

	#markerTargets = np.vstack((lhTargets, rhTargets, llTargets, tTargets, rlTargets))
	markerTargets = np.vstack((lhTargets, rhTargets))

	# In[95]:


	## Shuffle the data
	Data_CSP, markerTargets_CSP = shuffle(Data_CSP, markerTargets, random_state=0)
	## Split into train and test sets
	DataTrain_CSP, DataTest_CSP, markerTargetsTrain_CSP, markerTargetsTest_CSP = train_test_split(Data_CSP, markerTargets_CSP, test_size=0.3, random_state=1)

	##Use the same model to train and test
	history_CSP = LSTM_EEG.fit(DataTrain_CSP, markerTargetsTrain_CSP, epochs=30,verbose=2, batch_size=16)


	# In[96]:


	predictionsTest = LSTM_EEG.predict(DataTest_CSP)


	# In[97]:


	predictionsTest[predictionsTest>0.5] = 1


	# In[98]:


	predictionsTest[predictionsTest <= 0.5] = 0


	# In[99]:


	comparisonArrayTest = predictionsTest == markerTargetsTest


	# In[100]:


	correctCountTest = 0
	for boolValues in comparisonArrayTest:
		if(boolValues[0] & boolValues[1]):
			correctCountTest += 1
	falseCountTest = DataTest_CSP.shape[0] - correctCountTest

	predictionsTrain = LSTM_EEG.predict(DataTrain_CSP)
	predictionsTrain[predictionsTrain>0.5] = 1;
	predictionsTrain[predictionsTrain<=0.5] = 0;
	comparisonArrayTrain = predictionsTrain == markerTargetsTrain;

	correctCountTrain = 0
	for boolValues in comparisonArrayTrain:
		if(boolValues[0] & boolValues[1]):
			correctCountTrain += 1
	falseCountTrain = DataTrain_CSP.shape[0] - correctCountTrain

	#Computing the accuracy
	trainAcc_wCSP = (correctCountTrain*100/DataTrain_CSP.shape[0]);
	testAcc_wCSP = (correctCountTest*100/DataTest_CSP.shape[0]);

	#Save these values into the array
	trainAccuracyWithCSP[0,dataset] = trainAcc_wCSP;
	testAccuracyWithCSP[0, dataset] = testAcc_wCSP;

	print("#################################")
	print("### RESULTS AFTER APPLYING CSP ##")
	print("#################################")
	print("Training Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTrain, falseCountTrain, trainAcc_wCSP))
	print("#################################")
	print("#################################")
	print("Testing Performance:\nCorrect MI Prediction: {}\nIncorrect MI Prediction: {}\nPercent Accuracy: {:.3f}%".format(correctCountTest, falseCountTest, testAcc_wCSP))
	print("#################################")
	print("#################################")




[[ -0.    -0.    -0.   ...  23.8   10.74   0.76]
 [ -0.    -0.    -0.   ... -28.4  -37.39 -47.95]
 [ -0.    -0.    -0.   ...   4.31   5.51   3.66]
 ...
 [ -0.    -0.    -0.   ...   6.99   5.57   3.68]
 [ -0.    -0.    -0.   ...  -8.31  -9.34  -7.32]
 [ -0.    -0.    -0.   ...  -6.    -5.99  -4.9 ]]
202
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 21, 100)           121200    
_________________________________________________________________
dropout (Dropout)            (None, 21, 100)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                30200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense (Dense)   

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:93: ComplexWarning: Casting complex values to real discards the imaginary part


27/27 - 0s - loss: 0.8706 - accuracy: 0.6493
Epoch 2/30
27/27 - 0s - loss: 0.4301 - accuracy: 0.8104
Epoch 3/30
27/27 - 0s - loss: 0.2908 - accuracy: 0.8839
Epoch 4/30
27/27 - 0s - loss: 0.2461 - accuracy: 0.8886
Epoch 5/30
27/27 - 0s - loss: 0.2246 - accuracy: 0.9218
Epoch 6/30
27/27 - 0s - loss: 0.2286 - accuracy: 0.9100
Epoch 7/30
27/27 - 0s - loss: 0.1957 - accuracy: 0.9171
Epoch 8/30
27/27 - 0s - loss: 0.1815 - accuracy: 0.9147
Epoch 9/30
27/27 - 0s - loss: 0.1499 - accuracy: 0.9479
Epoch 10/30
27/27 - 0s - loss: 0.1490 - accuracy: 0.9431
Epoch 11/30
27/27 - 0s - loss: 0.1423 - accuracy: 0.9479
Epoch 12/30
27/27 - 0s - loss: 0.1289 - accuracy: 0.9360
Epoch 13/30
27/27 - 0s - loss: 0.1465 - accuracy: 0.9431
Epoch 14/30
27/27 - 0s - loss: 0.1225 - accuracy: 0.9573
Epoch 15/30
27/27 - 0s - loss: 0.1255 - accuracy: 0.9621
Epoch 16/30
27/27 - 0s - loss: 0.1361 - accuracy: 0.9431
Epoch 17/30
27/27 - 0s - loss: 0.1193 - accuracy: 0.9550
Epoch 18/30
27/27 - 0s - loss: 0.1389 - accuracy: 0

28/28 - 1s - loss: 0.7011 - accuracy: 0.5125
Epoch 2/30
28/28 - 1s - loss: 0.6437 - accuracy: 0.6629
Epoch 3/30
28/28 - 1s - loss: 0.5967 - accuracy: 0.7062
Epoch 4/30
28/28 - 1s - loss: 0.5297 - accuracy: 0.7494
Epoch 5/30
28/28 - 1s - loss: 0.4554 - accuracy: 0.7995
Epoch 6/30
28/28 - 1s - loss: 0.4024 - accuracy: 0.8223
Epoch 7/30
28/28 - 1s - loss: 0.4228 - accuracy: 0.8223
Epoch 8/30
28/28 - 0s - loss: 0.3047 - accuracy: 0.8747
Epoch 9/30
28/28 - 1s - loss: 0.2816 - accuracy: 0.8747
Epoch 10/30
28/28 - 1s - loss: 0.2730 - accuracy: 0.8975
Epoch 11/30
28/28 - 1s - loss: 0.2330 - accuracy: 0.9134
Epoch 12/30
28/28 - 1s - loss: 0.2269 - accuracy: 0.9066
Epoch 13/30
28/28 - 1s - loss: 0.2723 - accuracy: 0.8884
Epoch 14/30
28/28 - 1s - loss: 0.2942 - accuracy: 0.8610
Epoch 15/30
28/28 - 1s - loss: 0.2257 - accuracy: 0.8975
Epoch 16/30
28/28 - 1s - loss: 0.2407 - accuracy: 0.8952
Epoch 17/30
28/28 - 1s - loss: 0.2123 - accuracy: 0.9157
Epoch 18/30
28/28 - 1s - loss: 0.1601 - accuracy: 0

Epoch 11/30
28/28 - 1s - loss: 0.3258 - accuracy: 0.8405
Epoch 12/30
28/28 - 1s - loss: 0.2765 - accuracy: 0.8747
Epoch 13/30
28/28 - 1s - loss: 0.2908 - accuracy: 0.8747
Epoch 14/30
28/28 - 1s - loss: 0.2575 - accuracy: 0.8884
Epoch 15/30
28/28 - 1s - loss: 0.2752 - accuracy: 0.8747
Epoch 16/30
28/28 - 1s - loss: 0.2910 - accuracy: 0.8747
Epoch 17/30
28/28 - 1s - loss: 0.2203 - accuracy: 0.9089
Epoch 18/30
28/28 - 1s - loss: 0.1933 - accuracy: 0.9180
Epoch 19/30
28/28 - 1s - loss: 0.2582 - accuracy: 0.8952
Epoch 20/30
28/28 - 1s - loss: 0.2490 - accuracy: 0.8929
Epoch 21/30
28/28 - 1s - loss: 0.2494 - accuracy: 0.8975
Epoch 22/30
28/28 - 0s - loss: 0.2514 - accuracy: 0.8952
Epoch 23/30
28/28 - 1s - loss: 0.2883 - accuracy: 0.8610
Epoch 24/30
28/28 - 0s - loss: 0.2228 - accuracy: 0.9021
Epoch 25/30
28/28 - 0s - loss: 0.2637 - accuracy: 0.8907
Epoch 26/30
28/28 - 0s - loss: 0.2156 - accuracy: 0.9112
Epoch 27/30
28/28 - 0s - loss: 0.1897 - accuracy: 0.9271
Epoch 28/30
28/28 - 0s - loss: 

...

Epoch 11/30
28/28 - 1s - loss: 0.3089 - accuracy: 0.8542
Epoch 12/30
28/28 - 1s - loss: 0.2616 - accuracy: 0.8975
Epoch 13/30
28/28 - 1s - loss: 0.2594 - accuracy: 0.8929
Epoch 14/30
28/28 - 1s - loss: 0.2342 - accuracy: 0.8998
Epoch 15/30
28/28 - 1s - loss: 0.2125 - accuracy: 0.9066
Epoch 16/30
28/28 - 1s - loss: 0.2517 - accuracy: 0.8929
Epoch 17/30
28/28 - 1s - loss: 0.2143 - accuracy: 0.9157
Epoch 18/30
28/28 - 1s - loss: 0.2417 - accuracy: 0.9157
Epoch 19/30
28/28 - 1s - loss: 0.1781 - accuracy: 0.9339
Epoch 20/30
28/28 - 1s - loss: 0.2230 - accuracy: 0.9021
Epoch 21/30
28/28 - 1s - loss: 0.2057 - accuracy: 0.9134
Epoch 22/30
28/28 - 1s - loss: 0.2065 - accuracy: 0.9112
Epoch 23/30
28/28 - 1s - loss: 0.1681 - accuracy: 0.9408
Epoch 24/30
28/28 - 1s - loss: 0.1404 - accuracy: 0.9385
Epoch 25/30
28/28 - 1s - loss: 0.1746 - accuracy: 0.9362
Epoch 26/30
28/28 - 1s - loss: 0.1835 - accuracy: 0.9271
Epoch 27/30
28/28 - 1s - loss: 0.1750 - accuracy: 0.9408
Epoch 28/30
28/28 - 1s - loss: 

28/28 - 1s - loss: 0.6932 - accuracy: 0.5376
Epoch 2/30
28/28 - 1s - loss: 0.5884 - accuracy: 0.7221
Epoch 3/30
28/28 - 1s - loss: 0.4691 - accuracy: 0.7813
Epoch 4/30
28/28 - 1s - loss: 0.4890 - accuracy: 0.7722
Epoch 5/30
28/28 - 1s - loss: 0.3564 - accuracy: 0.8497
Epoch 6/30
28/28 - 1s - loss: 0.2823 - accuracy: 0.8952
Epoch 7/30
28/28 - 1s - loss: 0.2859 - accuracy: 0.8679
Epoch 8/30
28/28 - 1s - loss: 0.2625 - accuracy: 0.8861
Epoch 9/30
28/28 - 1s - loss: 0.2528 - accuracy: 0.8929
Epoch 10/30
28/28 - 1s - loss: 0.2349 - accuracy: 0.9134
Epoch 11/30
28/28 - 1s - loss: 0.2811 - accuracy: 0.8793
Epoch 12/30
28/28 - 1s - loss: 0.1870 - accuracy: 0.9226
Epoch 13/30
28/28 - 1s - loss: 0.1986 - accuracy: 0.9271
Epoch 14/30
28/28 - 1s - loss: 0.1863 - accuracy: 0.9134
Epoch 15/30
28/28 - 1s - loss: 0.1857 - accuracy: 0.9431
Epoch 16/30
28/28 - 1s - loss: 0.1278 - accuracy: 0.9544
Epoch 17/30
28/28 - 1s - loss: 0.1286 - accuracy: 0.9385
Epoch 18/30
28/28 - 1s - loss: 0.2077 - accuracy: 0

Epoch 11/30
28/28 - 0s - loss: 0.1574 - accuracy: 0.9294
Epoch 12/30
28/28 - 0s - loss: 0.1328 - accuracy: 0.9499
Epoch 13/30
28/28 - 0s - loss: 0.1185 - accuracy: 0.9658
Epoch 14/30
28/28 - 1s - loss: 0.1011 - accuracy: 0.9613
Epoch 15/30
28/28 - 1s - loss: 0.0826 - accuracy: 0.9727
Epoch 16/30
28/28 - 1s - loss: 0.0847 - accuracy: 0.9727
Epoch 17/30
28/28 - 1s - loss: 0.0948 - accuracy: 0.9636
Epoch 18/30
28/28 - 1s - loss: 0.0882 - accuracy: 0.9727
Epoch 19/30
28/28 - 1s - loss: 0.0942 - accuracy: 0.9727
Epoch 20/30
28/28 - 1s - loss: 0.0734 - accuracy: 0.9795
Epoch 21/30
28/28 - 1s - loss: 0.0648 - accuracy: 0.9772
Epoch 22/30
28/28 - 1s - loss: 0.0546 - accuracy: 0.9818
Epoch 23/30
28/28 - 1s - loss: 0.0785 - accuracy: 0.9727
Epoch 24/30
28/28 - 1s - loss: 0.0457 - accuracy: 0.9863
Epoch 25/30
28/28 - 1s - loss: 0.0465 - accuracy: 0.9795
Epoch 26/30
28/28 - 1s - loss: 0.0270 - accuracy: 0.9932
Epoch 27/30
28/28 - 1s - loss: 0.0545 - accuracy: 0.9886
Epoch 28/30
28/28 - 1s - loss: 

42/42 - 1s - loss: 0.7071 - accuracy: 0.5022
Epoch 2/30
42/42 - 1s - loss: 0.6557 - accuracy: 0.6222
Epoch 3/30
42/42 - 1s - loss: 0.6046 - accuracy: 0.7091
Epoch 4/30
42/42 - 1s - loss: 0.5717 - accuracy: 0.7181
Epoch 5/30
42/42 - 1s - loss: 0.5502 - accuracy: 0.7046
Epoch 6/30
42/42 - 1s - loss: 0.4776 - accuracy: 0.8006
Epoch 7/30
42/42 - 1s - loss: 0.4963 - accuracy: 0.7376
Epoch 8/30
42/42 - 1s - loss: 0.4600 - accuracy: 0.7916
Epoch 9/30
42/42 - 1s - loss: 0.4229 - accuracy: 0.8006
Epoch 10/30
42/42 - 1s - loss: 0.4106 - accuracy: 0.8036
Epoch 11/30
42/42 - 1s - loss: 0.3997 - accuracy: 0.8156
Epoch 12/30
42/42 - 1s - loss: 0.3650 - accuracy: 0.8246
Epoch 13/30
42/42 - 1s - loss: 0.3737 - accuracy: 0.8456
Epoch 14/30
42/42 - 1s - loss: 0.3107 - accuracy: 0.8606
Epoch 15/30
42/42 - 1s - loss: 0.3291 - accuracy: 0.8591
Epoch 16/30
42/42 - 1s - loss: 0.2770 - accuracy: 0.8891
Epoch 17/30
42/42 - 1s - loss: 0.2851 - accuracy: 0.8831
Epoch 18/30
42/42 - 1s - loss: 0.2973 - accuracy: 0

Epoch 11/30
27/27 - 1s - loss: 0.1904 - accuracy: 0.9229
Epoch 12/30
27/27 - 1s - loss: 0.1524 - accuracy: 0.9346
Epoch 13/30
27/27 - 0s - loss: 0.1751 - accuracy: 0.9182
Epoch 14/30
27/27 - 1s - loss: 0.1417 - accuracy: 0.9509
Epoch 15/30
27/27 - 1s - loss: 0.1605 - accuracy: 0.9439
Epoch 16/30
27/27 - 1s - loss: 0.2037 - accuracy: 0.9159
Epoch 17/30
27/27 - 0s - loss: 0.1530 - accuracy: 0.9416
Epoch 18/30
27/27 - 0s - loss: 0.1310 - accuracy: 0.9439
Epoch 19/30
27/27 - 1s - loss: 0.1220 - accuracy: 0.9579
Epoch 20/30
27/27 - 1s - loss: 0.1134 - accuracy: 0.9556
Epoch 21/30
27/27 - 0s - loss: 0.1320 - accuracy: 0.9486
Epoch 22/30
27/27 - 1s - loss: 0.1150 - accuracy: 0.9650
Epoch 23/30
27/27 - 1s - loss: 0.1135 - accuracy: 0.9463
Epoch 24/30
27/27 - 1s - loss: 0.1014 - accuracy: 0.9603
Epoch 25/30
27/27 - 1s - loss: 0.0639 - accuracy: 0.9860
Epoch 26/30
27/27 - 1s - loss: 0.0948 - accuracy: 0.9743
Epoch 27/30
27/27 - 1s - loss: 0.1300 - accuracy: 0.9509
Epoch 28/30
27/27 - 1s - loss: 

In [2]:
fileNames

['CLASubjectA1601083StLRHand.mat',
 'CLASubjectB1510193StLRHand.mat',
 'CLASubjectB1510203StLRHand.mat',
 'CLASubjectB1512153StLRHand.mat',
 'CLASubjectC1511263StLRHand.mat',
 'CLASubjectC1512163StLRHand.mat',
 'CLASubjectC1512233StLRHand.mat',
 'CLASubjectD1511253StLRHand.mat',
 'CLASubjectE1512253StLRHand.mat',
 'CLASubjectE1601193StLRHand.mat',
 'CLASubjectE1601223StLRHand.mat',
 'CLASubjectF1509163StLRHand.mat',
 'CLASubjectF1509173StLRHand.mat',
 'CLASubjectF1509283StLRHand.mat']

In [3]:
testAccuracyNoCSP

array([[62.63736264, 50.26455026, 50.26455026, 55.55555556, 63.49206349,
        61.9047619 , 65.07936508, 48.93617021, 64.55026455, 60.31746032,
        70.74468085, 54.70383275, 50.54347826, 54.45026178]])

In [4]:
testAccuracyWithCSP

array([[86.81318681, 85.18518519, 84.12698413, 78.30687831, 88.35978836,
        89.41798942, 85.71428571, 83.5106383 , 86.24338624, 92.06349206,
        95.21276596, 93.03135889, 88.58695652, 88.48167539]])

In [5]:
trainAccuracyNoCSP

array([[100.        ,  74.94305239,  94.76082005,  99.77220957,
        100.        ,  98.86104784, 100.        ,  97.26027397,
         99.3166287 ,  99.77220957,  99.543379  ,  96.4017991 ,
         93.92523364,  98.41986456]])

In [6]:
trainAccuracyWithCSP

array([[100.        ,  97.72209567, 100.        ,  98.40546697,
         99.08883827,  99.54441913,  99.08883827,  99.7716895 ,
         99.3166287 , 100.        , 100.        ,  99.55022489,
         99.53271028,  98.19413093]])

In [7]:
combinedArray = np.transpose(np.asarray(np.vstack((trainAccuracyNoCSP,testAccuracyNoCSP,trainAccuracyWithCSP, testAccuracyWithCSP))))

In [8]:
combinedArray.shape

(14, 4)

In [10]:
np.savetxt('CLAperSubject_Dropout.csv', combinedArray, delimiter=',')